In [3]:
import asyncio  
import time  
from datetime import datetime

async def custom_sleep():  
    print('SLEEP {}\n'.format(datetime.now()))
    await asyncio.sleep(1)

async def factorial(name, number):  
    f = 1
    for i in range(2, number+1):
        print('Task {}: Compute factorial({})'.format(name, i))
        await custom_sleep()
        f *= i
    print('Task {}: factorial({}) is {}\n'.format(name, number, f))

start = time.time()  
loop = asyncio.get_event_loop()
tasks = [  
    asyncio.ensure_future(factorial("A", 3)),
    asyncio.ensure_future(factorial("B", 4)),
]
# loop.run_until_complete(asyncio.wait(tasks))  
# loop.close()
end = time.time()  
print("Total time: {}".format(end - start))

Total time: 0.0009937286376953125


Task A: Compute factorial(2)
SLEEP 2023-02-26 14:35:57.740231

Task B: Compute factorial(2)
SLEEP 2023-02-26 14:35:57.740231

Task A: Compute factorial(3)
SLEEP 2023-02-26 14:35:58.751072

Task B: Compute factorial(3)
SLEEP 2023-02-26 14:35:58.751072

Task A: factorial(3) is 6

Task B: Compute factorial(4)
SLEEP 2023-02-26 14:35:59.753418

Task B: factorial(4) is 24



In [4]:
import asyncio

async def sleep_5():
    print('Sleeping_5')
    await asyncio.sleep(5)
    print('GM_5')
    
async def sleep_3():
    print('Sleeping_3')
    await asyncio.sleep(3)
    print('GM_3')
    
    
loop = asyncio.get_event_loop()
await asyncio.gather(sleep_3(), sleep_5())
print('asdads')

Sleeping_3
Sleeping_5
GM_3
GM_5
asdads


# Python Basics

In [1]:
l = lambda x, y: print(x * y)
l(1,3)

3


In [9]:
a = [(10,8), (1,2), (3,4), (2,6)]
a.sort(key=lambda x: x[0])
a

[(1, 2), (2, 6), (3, 4), (10, 8)]

In [43]:
list1 = [5, 88, 12]
list2 = [6,9,8]
data = list(zip(list1, list2))
data.sort()
list1, list2 = map(lambda t: list(t), zip(*data))

In [42]:
list3 = ('apple aple appple', 'cyber', 'quakc')

def do_smth(n):
    return len(n)

x = list(map(do_smth, list3))
print(x)

[17, 5, 5]


In [34]:
from pprint import pprint
my_dict = {'name': 'Yasoob', 'age': 'undefined', 'personality': 'awesome', 'mood':'hungry'}

{'name': 'Yasoob',
 'age': 'undefined',
 'personality': 'awesome',
 'mood': 'hungry'}

In [37]:
from functools import lru_cache

AttributeError: module 'json' has no attribute 'tool'

# Context managers

In [ ]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        return self.file_obj
    
    def __exit__(self, type, value, traceback):
        print("Exception was handled")
        self.file_obj.close()
        return True
    
with File('some_file', 'w') as f:
    f.write('hola!')

In [41]:
from contextlib import contextmanager

@contextmanager
def open_file(name):
    f = open(name, 'w')
    yield f
    f.close()
    
with open_file('some_file') as f:
    f.write('hola!')

# Serialize

In [10]:
from marshmallow import Schema, fields

class Note:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        
        def __repr__(self):
            return "<User(name={self.name!r})>".format(self=self)
            
class NoteSchema(Schema):
    name = fields.Str()
    description = fields.Str()
    
note = Note(name="just note", description="some desq")
schema = NoteSchema()
result = schema.dump(note)
print(result)

json = '{"name":"another note", "description":"note from json"}'
result = schema.loads(json)
print(result)

result = schema.load({'name': 'yet another note', 'description': 'from dict'})
print(result)

{'name': 'just note', 'description': 'some desq'}
{'name': 'another note', 'description': 'note from json'}
{'name': 'yet another note', 'description': 'from dict'}


In [14]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship, backref


engine = sa.create_engine("sqlite:///:memory:")
session = scoped_session(sessionmaker(bind=engine))
Base = declarative_base()


class Author(Base):
    __tablename__ = "authors"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String, nullable=False)

    def __repr__(self):
        return "<Author(name={self.name!r})>".format(self=self)


class Book(Base):
    __tablename__ = "books"
    id = sa.Column(sa.Integer, primary_key=True)
    title = sa.Column(sa.String)
    author_id = sa.Column(sa.Integer, sa.ForeignKey("authors.id"))
    author = relationship("Author", backref=backref("books"))


Base.metadata.create_all(engine)

from marshmallow_sqlalchemy import SQLAlchemySchema, auto_field


class AuthorSchema(SQLAlchemySchema):
    class Meta:
        model = Author
        load_instance = True  # Optional: deserialize to model instances

    id = auto_field()
    name = auto_field()


class BookSchema(SQLAlchemySchema):
    class Meta:
        model = Book
        load_instance = True

    id = auto_field()
    title = auto_field()
    
author = Author(name="Chuck Paluhniuk")
author_schema = AuthorSchema()
book = Book(title="Fight Club", author=author)
session.add(author)
session.add(book)
session.commit()

dump_data = author_schema.dump(author)
print(dump_data)
# {'id': 1, 'name': 'Chuck Paluhniuk', 'books': [1]}

load_data = author_schema.load(dump_data, session=session)
print(load_data)
# <Author(name='Chuck Paluhniuk')>

{'name': 'Chuck Paluhniuk', 'id': 1}
<Author(name='Chuck Paluhniuk')>


# Plotly

In [4]:
JSON_PATH = r"C:\Users\serge\AppData\Local\Google\Chrome\User Data\Default\Bookmarks"

import json

data = None

with open(JSON_PATH, 'r', encoding='utf-8') as f:
    data = f.read()

rdata = json.loads(data)

urls=[]

def walk(urls, childs):
    for child in childs['children']:
        if child['type'] == 'url':
            urls.append(child['url'])
        elif child['type'] == 'folder':
            walk(urls, child)

for root in rdata['roots']:
    for child in rdata['roots'][root]:
        walk(urls, rdata['roots'][root])
        
urls = list(dict.fromkeys(urls))

from urllib.parse import urlparse

counter = {}
EXCLUDE = [
    #'habr.com'
]
COUNT_LIMIT = 10


for url in urls:
    host = urlparse(url).netloc
#     host = host.split('.')[-1]
    if host not in counter:
        counter[host] = 0
    counter[host] += 1

import pandas as pd

df = pd.DataFrame(columns=['address','count'])
for i, key in enumerate(counter):
    value = counter[key]
    if value > COUNT_LIMIT and key not in EXCLUDE:
        df.loc[i] = [key, value]

import plotly.express as px
fig = px.bar(df, x='address', y='count')
fig.show()

Exception ignored in: <coroutine object wait at 0x00000221A431BC40>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object wait at 0x00000221A431BC40>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'


# Google note position

In [17]:
def print_format(num):
    formated = format(num, '#064b')
    print(str(formated) + '  |  ' + str(num))

def print_format_list(num_list):
    for i in num_list:
        print_format(i)

In [27]:
nums1 = [
    -562949952897025,
    2097152
]

nums2 = [
    -562949953421313,
    524288,
    2097152
]

nums3 = [
    -562949954469889,
    -562949953945601,
    -281474976710657,
    1048576
]

nums4 = [
    -562949955518465,
    -844424931180545,
    -562949954994177,
    -562949953945601,
    -281474976710657,
    1048576
]

nums5 = [
    -562949956567041,
    -844424931704833,
    -562949956042753,
    -562949954994177,
    -562949953945601,
    -281474976710657,
    1048576
]

In [28]:
print('count:' + str(1))
print_format_list(nums1)
print('count:' + str(2))
print_format_list(nums2)
print('count:' + str(3))
print_format_list(nums3)
print('count:' + str(4))
print_format_list(nums4)
print('count:' + str(5))
print_format_list(nums5)

count:1
-0b0000000000001111111111111111111111111111110000000000000000001  |  -562949952897025
0b00000000000000000000000000000000000000001000000000000000000000  |  2097152
count:2
-0b0000000000010000000000000000000000000000000000000000000000001  |  -562949953421313
0b00000000000000000000000000000000000000000010000000000000000000  |  524288
0b00000000000000000000000000000000000000001000000000000000000000  |  2097152
count:3
-0b0000000000010000000000000000000000000000100000000000000000001  |  -562949954469889
-0b0000000000010000000000000000000000000000010000000000000000001  |  -562949953945601
-0b0000000000001000000000000000000000000000000000000000000000001  |  -281474976710657
0b00000000000000000000000000000000000000000100000000000000000000  |  1048576
count:4
-0b0000000000010000000000000000000000000001000000000000000000001  |  -562949955518465
-0b0000000000011000000000000000000000000000100000000000000000001  |  -844424931180545
-0b00000000000100000000000000000000000000001100000000000000

# Telegram test bot

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import ParseMode

import logging
import os

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

def hello(update, context):
    context.bot.send_message(chat_id='@sakurano_ka_art',
                            text='*bold*', parse_mode=ParseMode.MARKDOWN_V2)
    update.message.reply_text('Sended')

def start(update, context):
    update.message.reply_text('Hello!')

def post(update, context):
    f = open("D:\Photo\image.jpg", 'rb')
    context.bot.send_photo(chat_id='@sakurano_ka_art', photo=f)    


def collback_half_minute(context):
    context.bot.send_message(chat_id='@sakurano_ka_art',
                            text='~BEEEP~', parse_mode=ParseMode.MARKDOWN_V2)
    

def handle_document(bot, update):
    chat_id = update.message.chat_id
    document = update.message.document
    print(document)
    reply_document(document=document)
#     context.bot.send_document(chat_id=chat_id)    
    
def handle_text(bot, update):
    reply_text(text='Here!') 
    

updater = Updater('992674991:AAH0q2z1W--MrqfDatIwjgba8E9Gqg76jLc', use_context=True)

#j = updater.job_queue
# job_half_minute = j.run_repeating(collback_half_minute, interval=30, first=0)

#updater.dispatcher.add_handler(CommandHandler('hello', hello))

dp = updater.dispatcher

# dp.add_handler(CommandHandler('start', start))
# dp.add_handler(CommandHandler('hello', hello))
# dp.add_handler(CommandHandler('post', post))
# dp.add_handler(CommandHandler('send', send))

document_handler = MessageHandler(Filters.document, handle_document)
text_handler = MessageHandler(Filters.text, handle_text)

dp.add_handler(document_handler)
dp.add_handler(text_handler)

updater.start_polling()
updater.idle()

 # Image verifier

In [ ]:
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGES_PATH = "F:\\"
BROKEN_PATH = 'F:\\BROKEN\\'
IMAGE_EXTS = {'jpg','gif','jpeg','png'}

broken_images = []

def walk_dir(path):
    images = []
    for root, dirs, files in os.walk(IMAGES_PATH):
        for f in files:
            if f.split('.')[-1].lower() in IMAGE_EXTS:
                images.append(os.path.join(root,f))
    return images


def is_valid_image(image_path):
    try:
        im = Image.open(image_path)
        #im.verify()
        return True
    except:
        return False
    
    
images = walk_dir(IMAGES_PATH)

for i in tqdm(images):
    if not is_valid_image(i):
        broken_images.append(i)
        
# for i in broken_images:
#     os.rename(i, os.path.join(BROKEN_PATH, i.split('\\')[-1]))